In [18]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [19]:
# evaluate multinomial logistic regression model
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.metrics import confusion_matrix

# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

import plotly.express as px
from IPython.display import display
import statsmodels.api as sm

<IPython.core.display.Javascript object>

In [20]:
data = pd.read_csv("Data/data_clean.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age"], axis=1)
Y = data["gender"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,Confidence in Success,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age
0,18,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,1,1,211,1,50
1,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,149,1,40
2,27,1,0,0,0,1,1,1,1,1,...,1,1,1,1,0,1,1,168,1,28
3,29,1,1,0,1,1,1,1,1,0,...,1,1,1,0,0,1,0,230,1,37
4,19,1,0,0,0,1,1,0,1,1,...,0,0,1,0,0,0,1,361,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10413,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,243,1,24
10414,10,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,197,1,39
10415,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,447,2,33
10416,12,0,0,1,0,0,1,0,1,0,...,1,1,0,0,0,0,0,167,1,24


<IPython.core.display.Javascript object>

In [21]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

# print(np.argmin(n_scores["test_score"]))

Mean Accuracy: 0.638 (+/- 0.014)


<IPython.core.display.Javascript object>

In [22]:
# for model in n_scores["estimator"]:
#     print(model.coef_)

# n_scores["estimator"].coef_

model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
5157,1,1
7580,1,1
4182,2,2
1207,2,2
5333,2,2
...,...,...
555,2,1
8497,2,2
8560,1,1
6252,2,2


,Questions,Male,Female,Other
0,Influence,0.526281,0.532785,-1.059066
1,Modesty,0.303302,0.177708,-0.481010
2,Daring,0.098691,0.147412,-0.246102
3,Confidence,-0.451176,-0.655913,1.107089
4,Ruling,0.363154,-0.124768,-0.238386
5,Responsibility,0.181918,0.031690,-0.213608
6,IntrovExtro,-0.103617,0.234001,-0.130384
7,Success,0.261294,0.315670,-0.576963
8,Humble,0.400186,0.301605,-0.701791
9,Leadership,0.223216,0.271903,-0.495119


<IPython.core.display.Javascript object>

In [23]:
# Using Statsmodels.api
# logit_model = sm.MNLogit(Y, X)
# result = logit_model.fit()
# print(result.summary2())

<IPython.core.display.Javascript object>

In [24]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Male", "Female", "Other"],
    y=["Male", "Female", "Other"],
)
fig.update_xaxes(side="top")
fig.show()

array([[1347,  413,    3],
       [ 697,  639,    3],
       [  11,    2,    0]], dtype=int64)

<IPython.core.display.Javascript object>

### Multinomial Log Reg with Age Group

In [25]:
data = pd.read_csv("Data/data_AgeGroup.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age", "Age_Group"], axis=1)
Y = data["Age_Group"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age,Age_Group
0,18,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,1,211,1,50,5.0
1,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,149,1,40,4.0
2,27,1,0,0,0,1,1,1,1,1,...,1,1,1,0,1,1,168,1,28,3.0
3,29,1,1,0,1,1,1,1,1,0,...,1,1,0,0,1,0,230,1,37,4.0
4,19,1,0,0,0,1,1,0,1,1,...,0,1,0,0,0,1,361,1,27,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10413,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,243,1,24,2.0
10414,10,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,197,1,39,4.0
10415,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,447,2,33,3.0
10416,12,0,0,1,0,0,1,0,1,0,...,1,0,0,0,0,0,167,1,24,2.0


<IPython.core.display.Javascript object>

In [30]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

Mean Accuracy: 0.315 (+/- 0.016)


<IPython.core.display.Javascript object>

In [28]:
model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = [
    "Questions",
    "Under 18",
    "18-24",
    "25-34",
    "35-44",
    "45-54",
    "55-64",
    "65+",
]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
5157,6.0,2.0
7580,6.0,2.0
4182,4.0,5.0
1207,5.0,3.0
5333,5.0,4.0
...,...,...
555,3.0,5.0
8497,2.0,2.0
8560,3.0,5.0
6252,2.0,2.0


,Questions,Under 18,18-24,25-34,35-44,45-54,55-64,65+
0,Influence,-0.471141,-0.037147,0.106591,0.302387,0.233766,0.110064,-0.244521
1,Modesty,0.278013,0.050401,0.034247,0.049567,-0.022215,-0.009111,-0.380902
2,Daring,0.411626,0.307896,0.277344,0.089426,-0.039425,-0.180162,-0.866705
3,Confidence,0.363375,-0.156995,-0.345404,-0.261665,-0.305617,0.173323,0.532982
4,Ruling,-0.078589,0.093683,0.235101,0.014987,-0.079831,-0.047458,-0.137893
5,Responsibility,0.277893,0.382507,0.243928,0.035842,-0.086745,-0.298205,-0.555221
6,IntrovExtro,0.002649,-0.158144,-0.049756,0.167311,0.010794,0.053621,-0.026475
7,Success,-0.084138,0.474414,0.423955,0.160856,-0.039429,-0.142971,-0.792687
8,Humble,-0.197479,0.037962,0.279037,0.061713,-0.001198,-0.056829,-0.123205
9,Leadership,-0.335812,0.055252,0.167576,0.100175,0.190869,0.093467,-0.271527


<IPython.core.display.Javascript object>

In [29]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Under 18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+",],
    y=["Under 18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+",],
)
fig.update_xaxes(side="top")
fig.show()

array([[  3, 101,  40,   6,   2,   0,   0],
       [ 13, 445, 292,  39,  32,   5,   4],
       [ 13, 371, 349,  63,  51,  14,   5],
       [  6, 171, 203,  75,  73,  13,   7],
       [ 10,  90, 128,  46, 102,  11,   5],
       [  7,  46,  75,  30,  78,  12,   2],
       [  1,   9,  23,   9,  31,   3,   1]], dtype=int64)

<IPython.core.display.Javascript object>

In [ ]:
data["Age_Group"].value_counts()